# ***xverse feature selection on aarp-ds-tu***

In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
%matplotlib inline
warnings.filterwarnings('ignore')
from platform import python_version
print(python_version())
import seaborn as sns

3.8.5


# reading the files

In [2]:
df = pd.read_csv('kunal_Response_Train.csv')
df.shape

(220658, 861)

In [3]:
df.drop(columns=['DS_MS458','DS_MS476'],axis=1,inplace=True)
df.shape

(220658, 859)

In [4]:
df1 = df
df1.shape

(220658, 859)

In [5]:
ls = []
rem = []
for i in df1.columns:
    if i.startswith('AARP_'):
        ls.append(i)
    elif i.startswith('DS_'):
        ls.append(i)
    elif i.startswith('TU_'):
        ls.append(i)
    elif i == 'RESPONSE_CNT' or i == 'CONVERSION_CNT' or i == 'SEQ_NO':
        ls.append(i)
    else:
        rem.append(i)
print(len(ls))
print(len(rem))

845
14


In [6]:
df2 = df1[ls]
df2.shape

(220658, 845)

In [7]:
df2['RESPONSE_CNT'].value_counts(dropna=False)

0.0    176636
1.0     44022
Name: RESPONSE_CNT, dtype: int64

# Removed vars list

In [8]:
ls = ['AARP_INSERT_DATE_d','TU_INSERT_DATE_d','DS_MS402','DS_MS403','AARP_LAST_UPDT_DT_d']
print('before shape:',df2.shape)
for i in ls:
    if i in df2.columns:
        df2.drop(i,axis=1,inplace=True)
print('after shape:',df2.shape)

before shape: (220658, 845)
after shape: (220658, 842)


In [9]:
df2['RESPONSE_CNT'].value_counts(dropna=False)

0.0    176636
1.0     44022
Name: RESPONSE_CNT, dtype: int64

---

In [10]:
dfresp70 = df2
dfresp70.shape

(220658, 842)

# for response model

In [11]:
aarpcols = []
dscols = []
tucols = []
for i in dfresp70.columns:
    if i.startswith('AARP_'):
        aarpcols.append(i)
    elif i.startswith('DS_'):
        dscols.append(i)
    elif i.startswith('TU_'):
        tucols.append(i)
print(len(aarpcols))
print(len(dscols))
print(len(tucols))

213
310
316


In [13]:
X = dfresp70[aarpcols]
Y = dfresp70['RESPONSE_CNT']

In [14]:
X.shape , Y.shape

((220658, 213), (220658,))

# Xverse

#  Voting Selector for variable selection

In [15]:
from xverse.ensemble import VotingSelector

In [16]:
X.shape , Y.shape

((220658, 213), (220658,))

In [ ]:
clf = VotingSelector(selection_techniques=['WOE','RF', 'ETC', 'CS', 'L_ONE'])
clf.fit(X, Y)

In [ ]:
clf.feature_importances_

In [ ]:
clf.feature_importances_.to_csv('aarp_feimp.csv',index=False)
clf.feature_importances_.shape

In [ ]:
clf.feature_votes_

In [ ]:
clf.feature_votes_.to_csv('aarp_votes.csv',index=False)
clf.feature_votes_.shape

# DS

In [12]:
X = dfresp70[dscols]
Y = dfresp70['RESPONSE_CNT']

In [13]:
X.shape , Y.shape

((220658, 310), (220658,))

# Xverse

#  Voting Selector for variable selection

In [14]:
from xverse.ensemble import VotingSelector

In [15]:
X.shape , Y.shape

((220658, 310), (220658,))

In [16]:
clf = VotingSelector(selection_techniques=['WOE','RF', 'ETC', 'CS', 'L_ONE'])
clf.fit(X, Y)

VotingSelector(no_of_features=155,
               selection_techniques=['WOE', 'RF', 'ETC', 'CS', 'L_ONE'])

In [17]:
clf.feature_importances_

Variable_Name  Information_Value  Random_Forest  Extra_Trees   Chi_Square  \
0        DS_MS520           0.112581       0.003679     0.006006   587.584526   
1        DS_MD006           0.111125       0.014635     0.007027   501.092040   
2        DS_MV039           0.076875       0.001551     0.002291  1606.926891   
3        DS_MT039           0.076598       0.000953     0.002324  1593.833462   
4        DS_MV007           0.074527       0.007780     0.004919   523.619422   
..            ...                ...            ...          ...          ...   
305      DS_DS984           0.000217       0.005242     0.003650    24.728648   
306      DS_DS986           0.000058       0.005678     0.003887     6.861907   
307      DS_DS961           0.000051       0.001845     0.002635     9.845849   
308      DS_DS784           0.000000       0.000000     0.000000          NaN   
309      DS_XB027           0.000000       0.000000     0.000000          NaN   

        L_One  
0   -0.002295  
1   -0.000479  
2    0.058758  
3    0.000000  
4   -0.001139  
..        ...  
305 -0.000054  
306 -0.000057  
307  0.000006  
308  0.000000  
309  0.000000  

[310 rows x 6 columns]

In [18]:
clf.feature_importances_.to_csv('ds_feimp.csv',index=False)
clf.feature_importances_.shape

(310, 6)

In [19]:
clf.feature_votes_

Variable_Name  Information_Value  Random_Forest  Extra_Trees  Chi_Square  \
0        DS_MS520                  1              1            1           1   
151      DS_MS518                  1              1            1           1   
47       DS_AU006                  1              1            1           1   
52       DS_VW105                  1              1            1           1   
59       DS_DS982                  1              1            1           1   
..            ...                ...            ...          ...         ...   
270      DS_XB807                  0              0            0           0   
265      DS_DS770                  0              0            0           0   
236      DS_XB017                  0              0            0           0   
171      DS_MT024                  0              0            0           0   
309      DS_XB027                  0              0            0           0   

     L_One  Votes  
0        1      5  
151      1      5  
47       1      5  
52       1      5  
59       1      5  
..     ...    ...  
270      0      0  
265      0      0  
236      0      0  
171      0      0  
309      0      0  

[310 rows x 7 columns]

In [20]:
clf.feature_votes_.to_csv('ds_votes.csv',index=False)
clf.feature_votes_.shape

(310, 7)

# TU

In [31]:
X = dfresp70[tucols]
Y = dfresp70['RESPONSE_CNT']

In [32]:
X.shape , Y.shape

((220658, 316), (220658,))

# Xverse

#  Voting Selector for variable selection

In [33]:
from xverse.ensemble import VotingSelector

In [34]:
X.shape , Y.shape

((220658, 316), (220658,))

In [35]:
clf = VotingSelector(selection_techniques=['WOE','RF', 'ETC', 'CS', 'L_ONE'])
clf.fit(X, Y)

VotingSelector(no_of_features=158,
               selection_techniques=['WOE', 'RF', 'ETC', 'CS', 'L_ONE'])

In [36]:
clf.feature_importances_

Variable_Name  Information_Value  Random_Forest  \
0         TU_NUM_INS_INQ_0TO24M           0.167295       0.013691   
1         TU_NUM_INS_INQ_0TO21M           0.157036       0.014057   
2         TU_NUM_INS_INQ_0TO18M           0.136604       0.006816   
3         TU_NUM_INS_INQ_0TO15M           0.128170       0.007413   
4         TU_NUM_INS_INQ_0TO12M           0.119506       0.007852   
..                          ...                ...            ...   
311       TU_ACCTS_W_MXD4_IN_3M           0.000000       0.000000   
312       TU_ACCTS_W_MXD4_IN_6M           0.000000       0.000000   
313  TU_ACCTS_CURR_90DAYPASTDUE           0.000000       0.000000   
314  TU_ACCTS_CURR_60DAYPASTDUE           0.000000       0.000000   
315        TU_NUM_INS_INQ_0TO1M           0.000000       0.000000   

     Extra_Trees   Chi_Square     L_One  
0       0.017542  3105.784606  0.057815  
1       0.015513  3023.676398  0.031409  
2       0.010956  2648.523090 -0.045491  
3       0.009982  2603.616277 -0.007578  
4       0.009411  2615.036819  0.033634  
..           ...          ...       ...  
311     0.000000          NaN  0.000000  
312     0.000000          NaN  0.000000  
313     0.000000          NaN  0.000000  
314     0.000000          NaN  0.000000  
315     0.000000          NaN  0.000000  

[316 rows x 6 columns]

In [37]:
clf.feature_importances_.to_csv('tu_feimp.csv',index=False)
clf.feature_importances_.shape

(316, 6)

In [38]:
clf.feature_votes_

Variable_Name  Information_Value  Random_Forest  \
0               TU_NUM_INS_INQ_0TO24M                  1              1   
73   TU_ACCTS_NEVER_120PLUSDAYPASTDUE                  1              1   
33              TU_ACCTS_MOS_OLD_OPEN                  1              1   
35    TU_PUBREC_TRDLNE_DEROG_GT100DOL                  1              1   
36         TU_BREV_ACCTS_MOS_OLD_OPEN                  1              1   
..                                ...                ...            ...   
247                     TU_PBANK_CARD                  0              0   
282                    TU_UPRET_ACCTS                  0              0   
284              TU_STUDENT_HC_RANGED                  0              0   
245     TU_BINSTALL_ACCTS_OPEN_0TO12M                  0              0   
315              TU_NUM_INS_INQ_0TO1M                  0              0   

     Extra_Trees  Chi_Square  L_One  Votes  
0              1           1      1      5  
73             1           1      1      5  
33             1           1      1      5  
35             1           1      1      5  
36             1           1      1      5  
..           ...         ...    ...    ...  
247            0           0      0      0  
282            0           0      0      0  
284            0           0      0      0  
245            0           0      0      0  
315            0           0      0      0  

[316 rows x 7 columns]

In [39]:
clf.feature_votes_.to_csv('tu_votes.csv',index=False)
clf.feature_votes_.shape

(316, 7)